<h1 align=center><font size = 5>Neighborhoods in the city of Toronto, CA </font></h1>

### Import the Libraries

In [2]:
import pandas as pd
import numpy as np

###### Check if required libraries are installed, else install

In [16]:
!pip install beautifulsoup4
!pip install lxml
!pip install html5lib
!pip install requests

In [3]:
from bs4 import BeautifulSoup
import requests
import csv

#### Data CReation in CSV from HTML page 

In [42]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(source, 'lxml')
    
#print(soup.prettify())

with open('toronto_Ca.csv','w') as csv_file:

    csv_writer = csv.writer(csv_file)
    #csv_writer.writerow(['Postcode','Borough','Neighbourhood'])
    
    tb = soup.find('table', class_='wikitable')

    for tr in tb.tbody:
        try:
            #print(i)
            str = tr.text
            str = str.replace('\n','--')
            str_list.extend(['',''])
            str_list = str.split("--")
            i = 0
            while  i <= str_list.count(''):
                str_list.remove('')
                i = i+1
            csv_writer.writerow(str_list)
        except Exception as e:
            pass
            #print('None')

    

###### Read the csv file into a dataframe

In [3]:
df = pd.read_csv('toronto_Ca.csv')
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


###### Remove the rows with borough that is Not assigned

In [4]:
df = df.replace({'Borough': 'Not assigned'},np.nan)
df.dropna(inplace=True)
df.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


Sort the Rows by 'Postcode'

In [5]:
df.sort_values(by=['Postcode'], inplace=True)
df.head()

,Postcode,Borough,Neighbourhood
11,M1B,Scarborough,Rouge
12,M1B,Scarborough,Malvern
29,M1C,Scarborough,Port Union
28,M1C,Scarborough,Rouge Hill
27,M1C,Scarborough,Highland Creek


Reset the index and check for multiple rows with same 'Postcode'

In [6]:
df.reset_index(drop=True, inplace=True)
dup = df.duplicated(subset= 'Postcode', keep='first')

#### Restructure the Dataframe

- Combine the different Neighbourhoods for same Postcode such that first occurrance of postcode contains all corresponding neighbourhoods,
- then remove the duplicate Postcodes rows and 
- reset the index.

In [7]:
for i in enumerate(dup):
    if(i[1] == True):
        j=1
        while (dup.loc[i[0]-j] == True):
            j = j+1
                
        df.loc[(i[0]-j),'Neighbourhood'] = df.loc[(i[0]-j),'Neighbourhood'] + ',' + df.loc[(i[0]),'Neighbourhood']
        
for d in enumerate(dup):
    if(d[1] == True):
        df.drop(d[0], axis=0, inplace=True)
df.reset_index(drop=True, inplace=True)
df.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Port Union,Rouge Hill,Highland Creek"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


- Replace the neighbourhood containing 'non assigned' value to corresponding Borough value

In [9]:
df.where(df != 'Not assigned', df['Borough'], inplace=True, axis=0)

#### print the final number of rows in the dataframe

In [14]:
print("The number of rows in the dataframe: ", df.shape[0])

The number of rows in the dataframe:  103


### THANK YOU !!!